In [1]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [1]:
from transformers import DebertaConfig, DebertaForMaskedLM

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
from transformers import AutoModel, AutoTokenizer, DebertaModel

In [5]:
tokenizer = AutoTokenizer.from_pretrained("tokenizer/deeptriage")

In [6]:
deberta_model = DebertaForMaskedLM.from_pretrained("microsoft/deberta-base")

Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import torch

In [8]:
from math import exp

In [10]:
train_file = "/home/mdafifal.mamun/notebooks/triagerX/notebook/data/deeptriage/deep_data.csv"

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv(train_file)

In [13]:
df.head()

,id,issue_id,issue_title,reported_time,owner,description
0,1,2,Testing if chromium id works,2008-08-30 16:00:21,NaN,what steps will reproduce the problem123what i...
1,2,3,This is a test,2008-08-31 02:47:11,NaN,product version 0214927urls if applicabl...
2,3,5,Java not working yet,2008-09-02 19:04:27,NaN,product version 0214927urls if applicabl...
3,4,6,Chrome treats links in frames differently than...,2008-09-02 19:08:01,NaN,product version 0214927 1583urls if appl...
4,5,7,Errors in importing from firefox,2008-09-02 19:08:46,venkataramana@chromium.org,product version 0214927urls if applicabl...


In [14]:
# Example text corpus
text = df.iloc[100].description

# Tokenize the text
tokenized_text = tokenizer.tokenize(text)

# Choose a random token to mask
num_mask = int(len(tokenized_text) * 0.15)
mask_indices = torch.randperm(len(tokenized_text))[:num_mask]
for masked_index in mask_indices:
    tokenized_text[masked_index] = tokenizer.mask_token

# Convert tokenized text to tensor
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])

# Predict masked tokens
with torch.no_grad():
    outputs = deberta_model(tokens_tensor)
    print(outputs)
    predictions = outputs.logits

perplexities = []
for masked_index in mask_indices:
    predicted_index = torch.argmax(predictions[0, masked_index]).item()
    predicted_prob = torch.softmax(predictions[0, masked_index], dim=0)[predicted_index].item()
    perplexity = exp(-predicted_prob)
    perplexities.append(perplexity)

average_perplexity = sum(perplexities) / len(perplexities)
print("Average Perplexity:", average_perplexity)

MaskedLMOutput(loss=None, logits=tensor([[[-0.7069,  1.0392,  2.0755,  ..., -0.5393, -0.8196,  0.8377],
         [-0.7034,  0.7181,  2.1707,  ..., -0.3309, -0.6374,  1.0890],
         [-0.7459,  0.0399,  0.8019,  ..., -0.4017, -0.8384,  0.6948],
         ...,
         [-0.2803,  0.4031,  1.8379,  ..., -0.3013, -0.7961,  0.9781],
         [-0.3269, -0.3824,  0.8291,  ..., -0.1651, -0.7697,  0.3243],
         [-0.1519,  0.0518,  0.9602,  ..., -0.2040, -0.7505,  0.6617]]]), hidden_states=None, attentions=None)
Average Perplexity: 0.9978077496510037


In [15]:
config = DebertaConfig(
    hidden_size = 1024,
    vocab_size= tokenizer.vocab_size,
    num_hidden_layers = 6,
    num_attention_heads = 12,
    intermediate_size = 1024,
    max_position_embeddings = 512
)

In [16]:
texts = [
    f"Issue Title: {row.issue_title} Issue Description: {row.description}"
    for _, row in df.iterrows()
]

In [23]:
import re

In [24]:
def clean_text(line):
    line = re.sub(r'-+',' ',line)
    line = re.sub(r'[^a-zA-Z, ]+'," ",line)
    line = re.sub(r'[ ]+'," ",line)
    line += "."
    return line

In [29]:
texts = [clean_text(f"{row.issue_title} {row.description}") for _, row in df.iterrows()]

In [27]:
texts[100]

'Issue Title Add option to disable spell check Issue Description product version see aboutversionurls if applicable other browsers testedadd ok or fail after other browsers where you have tested this issuesafari firefox ie what steps will reproduce the problem type something in a textbox in a language other then the one specified home language what is the expected resultdisplays text normallywhat happens insteadobviously will mark anything not in homelanguage as wrongplease provide any additional information below attach a screenshot if possible .'

In [17]:
training_corpus = (
    texts[i : i + 1000]
    for i in range(0, len(texts), 1000)
)

In [11]:
tokenized_texts = tokenizer(texts, truncation=True, padding=True)

In [18]:
tokenizer_trained= tokenizer.train_new_from_iterator(training_corpus, 20_000)
# tokenizer_trained = AutoTokenizer.from_pretrained("tokenizer/deeptriage")

In [32]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

In [35]:
tokens = tokenizer_trained(texts[100]).input_ids
[tokenizer_trained.decode(id) for id in tokens]

['[CLS]',
 'Add',
 ' option',
 ' to',
 ' disable',
 ' spell',
 ' check',
 ' product',
 ' version',
 ' see',
 ' aboutversionurls',
 ' if',
 ' applicable',
 ' other',
 ' browsers',
 ' testedadd',
 ' ok',
 ' or',
 ' fail',
 ' after',
 ' other',
 ' browsers',
 ' where',
 ' you',
 ' have',
 ' tested',
 ' this',
 ' issuesafari',
 ' firefox',
 ' ie',
 ' what',
 ' steps',
 ' will',
 ' reproduce',
 ' the',
 ' problem',
 ' type',
 ' something',
 ' in',
 ' a',
 ' textbox',
 ' in',
 ' a',
 ' language',
 ' other',
 ' then',
 ' the',
 ' one',
 ' specified',
 ' home',
 ' language',
 ' what',
 ' is',
 ' the',
 ' expected',
 ' resultdisplay',
 's',
 ' text',
 ' normallywhat',
 ' happens',
 ' instead',
 'ob',
 'vious',
 'ly',
 ' will',
 ' mark',
 ' anything',
 ' not',
 ' in',
 ' home',
 'language',
 ' as',
 ' wrong',
 'please',
 ' provide',
 ' any',
 ' additional',
 ' information',
 ' below',
 ' attach',
 ' a',
 ' screenshot',
 ' if',
 ' possible',
 '.',
 '[SEP]']

In [36]:
tokenizer_trained.save_pretrained("tokenizer/deeptriage")

('tokenizer/deeptriage/tokenizer_config.json',
 'tokenizer/deeptriage/special_tokens_map.json',
 'tokenizer/deeptriage/vocab.json',
 'tokenizer/deeptriage/merges.txt',
 'tokenizer/deeptriage/added_tokens.json',
 'tokenizer/deeptriage/tokenizer.json')

In [39]:
config = DebertaConfig(
    hidden_size = 1020,
    vocab_size= tokenizer.vocab_size,
    num_hidden_layers = 6,
    num_attention_heads = 12,
    intermediate_size = 1024,
    max_position_embeddings = 512
)

In [40]:
model = DebertaForMaskedLM(config=config)
print(model.num_parameters()) #10457864

90446769


In [41]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [42]:
df.head()

,id,issue_id,issue_title,reported_time,owner,description
0,1,2,Testing if chromium id works,2008-08-30 16:00:21,NaN,what steps will reproduce the problem123what i...
1,2,3,This is a test,2008-08-31 02:47:11,NaN,product version 0214927urls if applicabl...
2,3,5,Java not working yet,2008-09-02 19:04:27,NaN,product version 0214927urls if applicabl...
3,4,6,Chrome treats links in frames differently than...,2008-09-02 19:08:01,NaN,product version 0214927 1583urls if appl...
4,5,7,Errors in importing from firefox,2008-09-02 19:08:46,venkataramana@chromium.org,product version 0214927urls if applicabl...


In [73]:
df["text"] = [f"Issue Title: {row.issue_title}\tIssue Description: {row.description}" for _, row in df.iterrows()]

In [74]:
len(df)

163695

In [75]:
df = df[df.text.str.len() > 0]

In [76]:
len(df)

163695

In [77]:
df.head()

,id,issue_id,issue_title,reported_time,owner,description,text
0,1,2,Testing if chromium id works,2008-08-30 16:00:21,NaN,what steps will reproduce the problem123what i...,Issue Title: Testing if chromium id works\tIss...
1,2,3,This is a test,2008-08-31 02:47:11,NaN,product version 0214927urls if applicabl...,Issue Title: This is a test\tIssue Description...
2,3,5,Java not working yet,2008-09-02 19:04:27,NaN,product version 0214927urls if applicabl...,Issue Title: Java not working yet\tIssue Descr...
3,4,6,Chrome treats links in frames differently than...,2008-09-02 19:08:01,NaN,product version 0214927 1583urls if appl...,Issue Title: Chrome treats links in frames dif...
4,5,7,Errors in importing from firefox,2008-09-02 19:08:46,venkataramana@chromium.org,product version 0214927urls if applicabl...,Issue Title: Errors in importing from firefox\...


In [78]:

import torch
from torch.utils.data import Dataset
from accelerate import Accelerator, DistributedType

class LineByLineTextDataset(Dataset):
    def __init__(self, tokenizer, raw_datasets, max_length: int):
        self.padding = "max_length"
        self.text_column_name = 'text'
        self.max_length = max_length
        self.accelerator = Accelerator(gradient_accumulation_steps=1)
        self.tokenizer = tokenizer

        with self.accelerator.main_process_first():
            self.tokenized_datasets = raw_datasets.map(
                self.tokenize_function,
                batched=True,
                num_proc=4,
                remove_columns=[self.text_column_name],
                desc="Running tokenizer on dataset line_by_line",
            )
            self.tokenized_datasets.set_format('torch',columns=['input_ids'],dtype=torch.long)

    def tokenize_function(self,examples):
        examples[self.text_column_name] = [
            line for line in examples[self.text_column_name] if len(line[0]) > 0 and not line[0].isspace()
        ]
        return self.tokenizer(
            examples[self.text_column_name],
            padding=self.padding,
            truncation=True,
            max_length=self.max_length,
            return_special_tokens_mask=True,
        )
    def __len__(self):
        return len(self.tokenized_datasets)

    def __getitem__(self, i):
        return self.tokenized_datasets[i]

In [1]:
from sklearn.model_selection import train_test_split

In [80]:
X_train, y_train = train_test_split(df, test_size=0.2, shuffle=True)

In [81]:
from datasets import Dataset as HfDataset

In [82]:
train_dataset = HfDataset.from_pandas(X_train)
test_dataset = HfDataset.from_pandas(y_train)

In [83]:
train_line_dataset = LineByLineTextDataset(tokenizer_trained, train_dataset, 512)
test_line_dataset = LineByLineTextDataset(tokenizer_trained, test_dataset, 512)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Running tokenizer on dataset line_by_line (num_proc=4):   0%|          | 0/130956 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Running tokenizer on dataset line_by_line (num_proc=4):   0%|          | 0/32739 [00:00<?, ? examples/s]

In [92]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./model",
    overwrite_output_dir=True,
    # push_to_hub=True,
    # hub_model_id="Ransaka/sinhala-bert-yt",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    max_steps=5000,
    eval_steps=500,
    logging_steps=100,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    report_to='none',
    # hub_private_repo = True,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_line_dataset,
    eval_dataset= test_line_dataset, # change to your actual evaluation dataset
    )

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [93]:
trainer.train()

Step,Training Loss,Validation Loss
500,7.298600,7.290041
1000,7.086700,7.082200
1500,6.994800,6.943599
2000,6.859900,6.819908
2500,6.779300,6.732032
3000,6.740700,6.663119
3500,6.706000,6.620352
4000,6.600600,6.586898
4500,6.652000,6.569583
5000,6.651300,6.566217


Checkpoint destination directory ./model/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./model/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


TrainOutput(global_step=5000, training_loss=6.881377075195313, metrics={'train_runtime': 6017.7651, 'train_samples_per_second': 26.588, 'train_steps_per_second': 0.831, 'total_flos': 1.899695172225024e+16, 'train_loss': 6.881377075195313, 'epoch': 1.22})

In [ ]:
results = trainer.evaluate()

In [ ]:
import math

print(f">>> Perplexity: {math.exp(results['eval_loss']):.2f}")

>>> Perplexity: 1820.63


In [14]:
from tqdm import tqdm

In [15]:
with open("mlm_training_data.txt", "w") as f:
    for text in tqdm(texts):
        f.write(text + "\n")

  0%|          | 0/163695 [00:00<?, ?it/s]

100%|██████████| 163695/163695 [00:00<00:00, 508136.63it/s]


In [21]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="mlm_training_data.txt",
    block_size=4096,  # Choose an appropriate block size
)

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [22]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15,  # Probability of masking a token
)

In [23]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
training_args = TrainingArguments(
    output_dir="./roberta-mlm",
    overwrite_output_dir=True,
    num_train_epochs=10,  # Number of pre-training epochs
    per_device_train_batch_size=32,
    save_steps=1000,
    save_total_limit=2,
)

In [25]:
# Initialize Trainer for MLM pre-training
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Start pre-training for MLM
trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


RuntimeError: The expanded size of the tensor (726) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [32, 726].  Tensor sizes: [1, 514]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
